## RuCor to CoNLL-U

In [1]:
from corpuscula import Conllu
import csv
import difflib
import junky
from mordl import UposTagger, FeatsTagger, LemmaTagger
import os
import pandas as pd
#import random
import re
import textdistance
from toxine import TextPreprocessor
#from uuid import uuid4

#text_dist = textdistance.JaroWinkler().distance

#random.seed(42)  # for uuid

cdict_path = os.path.join('_models/upos-bert_model/cdict.pickle')

dataset_dir = '_dataset'
rucor_dir = os.path.join(dataset_dir, 'rucoref')
rucor_docs_fn = os.path.join(rucor_dir, 'Documents.txt')
rucor_groups_fn = os.path.join(rucor_dir, 'Groups.txt')
rucor_tokens_fn = os.path.join(rucor_dir, 'Tokens.txt')

log_fn = os.path.join(dataset_dir, 'out.log')

I1102 19:16:35.927671 139753066800960 wrapper.py:16] Loading dictionaries from /usr/local/lib/python3.6/dist-packages/pymorphy2_dicts/data
I1102 19:16:35.952976 139753066800960 wrapper.py:20] format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [2]:
tagger_u = UposTagger()
tagger_u.load('_models/upos-bert_model', device='cuda:0', dataset_device='cuda:0')
tagger_f = FeatsTagger()
tagger_f.load('_models/feats-bert_model', device='cuda:0', dataset_device='cuda:0')
#tagger_l = LemmaTagger()
#tagger_l.load('_models/lemma-ft_model', device='cuda:0', dataset_device='cuda:0')
tp = TextPreprocessor(cdict_restore_from=cdict_path)

Loading dataset... done.
Creating model... done.
Loading state_dict... done.
Fit corpus dict... done.
Loading dataset... done.
Creating model... done.
Loading state_dict... done.
Fit corpus dict... done.


Fit corpus dict... done.


In [3]:
docs = pd.read_csv(rucor_docs_fn, sep='\t', index_col='doc_id', quoting=csv.QUOTE_NONE)
groups = pd.read_csv(rucor_groups_fn, sep='\t', index_col='group_id', quoting=csv.QUOTE_NONE)

In [4]:
DOC_ID = 0
TAG_COREF_HEADS = False
NEED_SHIFT_ADJUST = True

In [5]:
def get_fns(doc_id):
    in_fn = os.path.join(rucor_dir, 'rucoref_texts', docs.loc[doc_id, 'path'])
    out_fn_ = os.path.join(dataset_dir, os.path.splitext(os.path.basename(in_fn))[0])
    out_ext_ = '.conllu'
    out_fn = out_fn_ + out_ext_
    return in_fn, out_fn

if DOC_ID:
    in_fn, out_fn = get_fns(DOC_ID)
    print(in_fn, out_fn, sep=', ')

In [6]:
def norm_punct(punct):
    return punct.replace('—', '-').replace(';', '.').replace('...', '.').replace('…', '.') \
                .replace('«', '"').replace('„', '"') \
                .replace('»', '"').replace('“', '"') \
                .replace('``', '"').replace("''", '"')

In [7]:
def get_raw(in_fn):
    re_html = re.compile('&[a-z]+;')

    with open(in_fn, 'rb') as f:
        raw = f.read().decode('utf-8-sig').lower()

        def process(match):
            text = match.group(0)
            len_text = len(text)
            text = tp._unescape_html(text)
            return ' ' * (len_text - len(text)) + text

        raw = re_html.sub(process, raw)

        raw_forms, raw_punct = [], []
        isalnum = None
        for ch in raw:
            res = ch.isalnum()
            if res:
                if res != isalnum:
                    raw_forms.append(ch)
                else:
                    raw_forms[-1] += ch
            elif not ch.isspace():
                raw_punct.append(ch)
            isalnum = res

        raw_forms_ids = []
        idx = 0
        for token in raw_forms:
            idx_ = raw.index(token, idx)
            raw_forms_ids.append(idx_)
            idx = idx_ + len(token)
        idx = 0
        raw_punct_ids = []
        for i, ch in enumerate(raw_punct):
            idx_ = raw.index(ch, idx)
            raw_punct_ids.append(idx_)
            raw_punct[i] = norm_punct(ch)
            idx = idx_ + 1

    return list(zip(raw_forms, raw_forms_ids)), list(zip(raw_punct, raw_punct_ids))

if DOC_ID:
    raw_corpus, raw_puncts = get_raw(in_fn)

In [8]:
# adjust correct shifts to the wrong ones in Rucor
def adjust_raw_corpus(doc_id, raw_corpus):
    for i, (form, idx) in enumerate(raw_corpus):
        if doc_id == 115:
            if idx >= 1288:
                raw_corpus[i] = form, idx + 2
            elif idx >= 771:
                raw_corpus[i] = form, idx + 1
        elif doc_id == 116:
            if idx >= 1545:
                raw_corpus[i] = form, idx + 7
            elif idx >= 884:
                raw_corpus[i] = form, idx + 6
            elif idx >= 858:
                raw_corpus[i] = form, idx + 5
            elif idx >= 394:
                raw_corpus[i] = form, idx + 4
            elif idx >= 388:
                raw_corpus[i] = form, idx + 3
            elif idx >= 386:
                raw_corpus[i] = form, idx + 2
            elif idx >= 165:
                raw_corpus[i] = form, idx + 1

if DOC_ID and NEED_SHIFT_ADJUST:
    adjust_raw_corpus(DOC_ID, raw_corpus)
    adjust_raw_corpus(DOC_ID, raw_puncts)

In [9]:
if DOC_ID:
    tp.clear_corpus()
    tp.load_pars(in_fn, eop=r'\n')

In [10]:
if DOC_ID:
    tp.do_all(tag_date=False, norm_punct=True)

In [11]:
if DOC_ID:
    _ = tp.save(out_fn + '$')

In [12]:
if DOC_ID:
    _ = tagger_f.predict(tagger_u.predict(out_fn + '$'), save_to=out_fn)

In [13]:
if DOC_ID:
    os.remove(out_fn + '$')

In [14]:
re_nonalnum = re.compile('(?:\W|_)+')
re_alnum = re.compile('(?:[^\W_])+')

if DOC_ID:
    corpus_orig = list(Conllu.load(out_fn))
    corpus = []

In [15]:
def make_corpus():
    tag_shortcut = tp.TAG_SHORTCUT[2:]
    masks = list(x[1:] for x in tp.TAG_MASKS.keys())
    for sent in corpus_orig:
        for tok in sent[0]:
            form, misc = tok['FORM'], tok['MISC']
            for misc_ in misc:
                if misc_ in masks:
                    form = misc[misc_]
                elif misc_ == tag_shortcut:
                    form = misc[misc_]
            corpus.append((re_nonalnum.sub('', form.lower()), misc, tok['UPOS'],
                           re_alnum.sub('', form)))

if DOC_ID:
    make_corpus()

In [16]:
def validate_corpus():
    corpus_, raw_corpus_ = re_nonalnum.sub('', ''.join(x[0] for x in corpus)), \
                           ''.join(x[0] for x in raw_corpus)
    if corpus_ != raw_corpus_:
        print('                CORPUS_:')
        print(corpus_)
        print('                RAW_CORPUS_:')
        print(raw_corpus_)
        raise ValueError('The corpus is not the same as the raw corpus!')

if DOC_ID:
    validate_corpus()

In [17]:
def process_corpus(corpus, raw_corpus):

    len_corpus, len_raw_corpus = len(corpus), len(raw_corpus)

    def find_next(i, j):
        form, misc = corpus[i][:2]
        if not form:
            return i + 1, j

        raw_form, raw_form_idx = raw_corpus[j]
        form_, raw_form_ = form, raw_form
        len_form, len_raw_form = len(form), len(raw_form)

        misc['Shift'] = str(raw_form_idx)

        i_, j_ = i + 1, j + 1
        if len_form < len_raw_form:
            while i_ < len_corpus and len(form_) < len_raw_form:
                form_ += corpus[i_][0]
                i_ += 1
            form = form_
        elif len_form > len_raw_form:
            while j_ < len_raw_corpus and len(raw_form_) < len_form:
                raw_form_ += raw_corpus[j_][0]
                j_ += 1
            raw_form = raw_form_

        if form != raw_form:
            raise ValueError('form [{}] is not equal to raw_form [{}]!'.format(form, raw_form))

        return i_, j_,

    mid_ = {'mid': 0}
    def get_mention_id():
        #mid = uuid.uuid4()
        mid_['mid'] += 1
        mid = mid_['mid']
        return str(mid)

    i = j = 0
    while i < len_corpus:
        i_, j_ = find_next(i, j)
        for ii in range(i, i_):  # TODO
            form, misc, upos = corpus[ii][:3]
            if TAG_COREF_HEADS and upos in ['NOUN', 'PRON', 'PROPN']:
                misc['Coref_' + get_mention_id()] = 'Head'
        i, j = i_, j_

if DOC_ID:
    process_corpus(corpus, raw_corpus)

In [18]:
def process_puncts(corpus, raw_puncts):

    sm = difflib.SequenceMatcher()
    len_corpus, len_raw_puncts = len(corpus), len(raw_puncts)

    def find_next(i, j):
        start = stop = 0
        for i in range(i, len_corpus):
            form, misc = corpus[i][:2]
            if not form:
                break
            shift = misc.get('Shift')
            if shift:
                start = int(shift)
        puncts, miscs = [], []
        for i_ in range(i, len_corpus):
            form, misc, _, punct = corpus[i_]
            if form:
                shift = misc.get('Shift')
                if shift:
                    stop = int(shift)
                    break
            else:
                puncts.append(norm_punct(punct))
                miscs.append(misc)
        if not stop:
            return len_corpus, j

        for j in range(j, len_raw_puncts):
            if raw_puncts[j][1] >= start:
                break
        for j_ in range(j, len_raw_puncts):
            if raw_puncts[j_][1] >= stop:
                break
        if j_ == j:
            i_, j_ = len_corpus, len_raw_puncts
        raws, shifts = zip(*raw_puncts[j:j_])
        for ir in range(len(raws) - 3):
            if raws[ir:ir + 3] == ('.',) * 3:
                raws = raws[:ir + 1] + ('', '') + raws[ir + 3:]

        sm.set_seqs(puncts, raws)
        matches = sm.get_matching_blocks()
        for a, b, size in matches:
            for k in range(size):
                miscs[a + k]['Shift'] = str(shifts[b + k])

        return i_, j_,

    i = j = 0
    while i < len_corpus:
        i, j = find_next(i, j)

if DOC_ID:
    process_puncts(corpus, raw_puncts)

In [19]:
if DOC_ID:
    Conllu.save(corpus_orig, out_fn)

In [20]:
if DOC_ID:
    corpus = list(Conllu.load(out_fn))

In [21]:
if DOC_ID:
    for sent in corpus:
        for tok in sent[0]:
            print('{:20s}{}'.format(tok['FORM'] or '', tok['MISC'].get('Shift', '')))

In [22]:
if DOC_ID:
    shifts = {}
    for sent in corpus:
        for tok in sent[0]:
            form, shift = tok['FORM'] or '', tok['MISC'].get('Shift', '')
            if shift:
                shifts[shift] = norm_punct(form)

In [23]:
if DOC_ID:
    raw_corpus

In [24]:
if DOC_ID:
    raw_puncts

In [25]:
if DOC_ID:
    for tks, tk_shifts in groups[groups['doc_id'] == DOC_ID][['content', 'tk_shifts']].values:
        tks = tks.split()
        tk_shifts = tk_shifts.split(',')
        assert len(tks) == len(tk_shifts), \
            'len({}) != len({})'.format(tks, tk_shifts)
        for tk, tk_shift in zip(tks, tk_shifts):
            tok = shifts.get(tk_shift)
            if tok is None:
                print('token {} ({}) is not found'.format(tk_shift, tk))
            elif norm_punct(tok) != norm_punct(tk):
                print('token {}: {} != {}'.format(tk_shift, tok, tk))

In [27]:
with open(log_fn, 'wt', encoding='utf-8') as f_log:

    def log(text=''):
        print(text)
        print(text, file=f_log)

    out_fn = os.path.join(dataset_dir, 'out.conllu')
    for doc_id in groups['doc_id'].unique():
        in_fn, _ = get_fns(doc_id)
        log('{}: {}'.format(doc_id, in_fn))

        raw_corpus, raw_puncts = get_raw(in_fn)
        if NEED_SHIFT_ADJUST:
            adjust_raw_corpus(doc_id, raw_corpus)
            adjust_raw_corpus(doc_id, raw_puncts)

        tp.clear_corpus()
        tp.load_pars(in_fn, eop=r'\n')
        tp.do_all(tag_date=False, norm_punct=True)
        tp.save(out_fn + '$')

        tagger_f.predict(tagger_u.predict(out_fn + '$'), save_to=out_fn)

        corpus_orig = list(Conllu.load(out_fn))
        corpus = []
        make_corpus()
        validate_corpus()
        process_corpus(corpus, raw_corpus)
        process_puncts(corpus, raw_puncts)
        Conllu.save(corpus_orig, out_fn)
        corpus = list(Conllu.load(out_fn))

        shifts = {}
        for sent in corpus:
            for tok in sent[0]:
                form, shift = tok['FORM'] or '', tok['MISC'].get('Shift', '')
                if shift:
                    shifts[shift] = norm_punct(form)
        for tks, tk_shifts in groups[groups['doc_id'] == doc_id][['content', 'tk_shifts']].values:
            tks = tks.split()
            tk_shifts = tk_shifts.split(',')
            assert len(tks) == len(tk_shifts), \
                'len({}) != len({})'.format(tks, tk_shifts)
            for tk, tk_shift in zip(tks, tk_shifts):
                tok = shifts.get(tk_shift)
                if tok is None:
                    print('token {} ({}) is not found'.format(tk_shift, tk))
                    log('token {} ({}) is not found'.format(tk_shift, tk))
                elif norm_punct(tok) != norm_punct(tk):
                    print('token {}: {} != {}'.format(tk_shift, tok, tk))
                    log('token {}: {} != {}'.format(tk_shift, tok, tk))
        log()

Load corpus... 

1: _dataset/rucoref/rucoref_texts/fiction/102_beliajev_nad_bezdnoj.txt
Load corpus
[=] 70                                                           
Corpus has been loaded: 70 sentences, 957 tokens
Processing corpus
  0%|          | 0/70 [00:00<?, ?it/s]

done.
Preprocess corpus
[=] 70                                                           
Corpus has been processed: 1 documents, 6 paragraphs, 70 sentences, 957 tokens
Save corpus
[=] 70                                                           
Corpus has been saved


100%|██████████| 70/70 [00:00<00:00, 181.87it/s]
Processing corpus
100%|██████████| 70/70 [00:00<00:00, 206.04it/s]

Load corpus
[=] 70                                                           
Corpus has been loaded: 70 sentences, 957 tokens
Save corpus
[=] 70                                                           
Corpus has been saved
Load corpus
[=] 70                                                           
Corpus has been loaded: 70 sentences, 957 tokens
Load corpus... 

...

Load corpus
[=] 41                                                           
Corpus has been loaded: 41 sentences, 724 tokens
Save corpus
[=] 41                                                           
Corpus has been saved
Load corpus
[=] 41                                                           
Corpus has been loaded: 41 sentences, 724 tokens
Load corpus... 

token 37: Санкт-Петербург != Санкт
token 37: Санкт-Петербург != Санкт
token 42 (-) is not found
token 42 (-) is not found
token 43 (Петербург) is not found
token 43 (Петербург) is not found
token 1175: Санкт-Петербурге != Санкт
token 1175: Санкт-Петербурге != Санкт
token 1180 (-) is not found
token 1180 (-) is not found
token 1181 (Петербурге) is not found
token 1181 (Петербурге) is not found
token 2241: Санкт-Петербург != Санкт
token 2241: Санкт-Петербург != Санкт
token 2246 (-) is not found
token 2246 (-) is not found
token 2247 (Петербург) is not found
token 2247 (Петербург) is not found

739: _dataset/rucoref/rucoref_texts/libru/yaglov.txt
Load corpus
[=] 49                                                           
Corpus has been loaded: 49 sentences, 519 tokens
Processing corpus
100%|██████████| 49/49 [00:00<00:00, 193887.64it/s]

done.
Preprocess corpus
[=] 49                                                           
Corpus has been processed: 1 documents, 26 paragraphs, 49 sentences, 519 tokens
Save corpus
[=] 49                                                           
Corpus has been saved



Processing corpus
100%|██████████| 49/49 [00:00<00:00, 301350.29it/s]



Load corpus
[=] 49                                                           
Corpus has been loaded: 49 sentences, 519 tokens
Save corpus
[=] 49                                                           
Corpus has been saved
Load corpus
[=] 49                                                           
Corpus has been loaded: 49 sentences, 519 tokens
